In [1]:
import pandas as pd


In [2]:
zips_Jay=pd.read_csv("/home/jian/SunnyD/Walmart_Kroger_Publix/Jay_3_brands_zip_within_10mile.csv",dtype=str)

In [3]:
zips_Jay.head(2)

,store_id,zip_code,lat,lng,zip_within_10_mile,number_of_dma,dma,dma2,dma3
0,walmart_2071,99515,61.14022809999999,-149.8699692,"['99519', '99518', '99515', '99514', '99517', ...",1,ANCHORAGE,NaN,NaN
1,walmart_2188,99577,61.3090716,-149.53553019999995,"['99577', '99505', '99521', '99567']",1,ANCHORAGE,NaN,NaN


In [4]:
DMA_zip=pd.read_excel("/home/jian/Docs/Geo_mapping/Zips by DMA by County16-17 nielsen.xlsx",dtype=str,skiprows=1)
DMA_zip=DMA_zip.iloc[:,[0,2]]
DMA_zip.columns=['zip_code','DMA']
DMA_zip_Dallas=DMA_zip[DMA_zip['DMA']=='DALLAS-FT. WORTH'].drop_duplicates()
DMA_zip_Mobile=DMA_zip[DMA_zip['DMA']=='MOBILE-PENSACOLA (FT WALT)'].drop_duplicates()

In [5]:
zips_Jay_Dallas=zips_Jay[zips_Jay['zip_code'].isin(DMA_zip_Dallas['zip_code'])]
zips_Jay_Dallas['DMA']='DALLAS-FT. WORTH'
zips_Jay_Mobile=zips_Jay[zips_Jay['zip_code'].isin(DMA_zip_Mobile['zip_code'])]
zips_Jay_Mobile['DMA']='MOBILE-PENSACOLA (FT WALT)'


/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.


In [7]:
len(DMA_zip_Dallas['zip_code'].unique())

588

In [7]:
def get_stores(df_input):
    df=df_input[['store_id','DMA','zip_within_10_mile']]
    df=df.reset_index()
    del df['index']
    
    df['brand']=df['store_id'].apply(lambda x: x.split("_")[0])
    df_agg=df.groupby(['brand','DMA'])['store_id'].count().to_frame().reset_index()
    df_agg=df_agg.rename(columns={"store_id":"store_count"})
    return df_agg

In [8]:
store_Dallas=get_stores(zips_Jay_Dallas)
store_Mobile=get_stores(zips_Jay_Mobile)


In [9]:
def get_zips(df_input):
    result=pd.DataFrame()
    df=df_input[['store_id','DMA','zip_within_10_mile']]
    df=df.reset_index()
    del df['index']
    df['zip_within_10_mile']=df['zip_within_10_mile'].apply(lambda x: x[1: len(x)-1].replace("'",''))
    df['zip_within_10_mile']=df['zip_within_10_mile'].apply(lambda x: x.replace(" ",''))
    
    for i in range(len(df)):
        zip_list=df['zip_within_10_mile'][i].split(',')
        df_zip_app=pd.DataFrame({"zip":zip_list,"DMA":[df['DMA'][0]]*len(zip_list),"Store_id":[df['store_id'][i]]*len(zip_list)},index=range(len(zip_list)))
        result=result.append(df_zip_app)
    result['brand']=result['Store_id'].apply(lambda x: x.split("_")[0])
    result['zip']=result['zip'].apply(lambda x: x.zfill(5))
    return result

In [10]:
zip_Dallas=get_zips(zips_Jay_Dallas)
zip_Mobile=get_zips(zips_Jay_Mobile)

unique_zips_Dallas=zip_Dallas[['DMA','zip']].drop_duplicates()
unique_zips_Mobile=zip_Mobile[['DMA','zip']].drop_duplicates()

In [11]:
writer=pd.ExcelWriter("/home/jian/SunnyD/Walmart_Kroger_Publix/SunnyD_zips_and_stores_in_Dallas_and_Mobile_DMA.xlsx",engine="xlsxwriter")
store_Dallas.to_excel(writer,'Dallas_store_counts',index=False)
store_Mobile.to_excel(writer,'Mobile_store_counts',index=False)

unique_zips_Dallas.to_excel(writer,'unique_zips_Dallas',index=False)
unique_zips_Mobile.to_excel(writer,'unique_zips_Mobile',index=False)

zip_Dallas.to_excel(writer,'zips_by_store_Dallas',index=False)
zip_Mobile.to_excel(writer,'zips_by_store_Mobile',index=False)
writer.save()